# <b> Real Time Stock Market Data Pipelines

## <b>Importing Required Libraries

In [6]:
import requests
from datetime import datetime
import pandas as pd
import sqlite3
import os

## <b>Extracting Data

In [7]:
API_KEY = 'cpr629hr01qifjjvhtrgcpr629hr01qifjjvhts0'
SYMBOLS = [
    "AAPL",   # Apple Inc.
    "MSFT",   # Microsoft Corporation
    "GOOGL",  # Alphabet Inc. (Google)
    "AMZN",   # Amazon.com Inc.
    "TSLA",   # Tesla Inc.
    "META",   # Meta Platforms Inc. (formerly Facebook)
    "NFLX",   # Netflix Inc.
    "NVDA",   # NVIDIA Corporation
    "INTC",   # Intel Corporation
    "AMD",    # Advanced Micro Devices Inc.
]

def fetch_quote(symbol):
    url = f'https://finnhub.io/api/v1/quote?symbol={symbol}&token={API_KEY}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if 'c' in data:
            return data
        else:
            print(f"Error fetching quote for {symbol}: {data}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error for {symbol}: {e}")
        return None

# Fetch quotes for all symbols
def extract_stock_data(symbols):
    all_quotes = {}
    for symbol in symbols:
        print(f"Fetching quote for {symbol}...")
        quote_data = fetch_quote(symbol)
        if quote_data:
            all_quotes[symbol] = quote_data
    df = pd.DataFrame.from_dict(all_quotes, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Symbol', 'c': 'Current_Price','d': 'Change','dp': 'Percent_Change', 'h': 'High_Price', 'l': 'Low_Price', 'o': 'Open_Price', 'pc': 'Previous_Close'}, inplace=True)
    df['timestamp'] = pd.Timestamp.now()
    return df

# Extract data
stock_data_df = extract_stock_data(SYMBOLS)
print(stock_data_df.head())
stock_data_df.to_csv('stock.csv')

Fetching quote for AAPL...
Fetching quote for MSFT...
Fetching quote for GOOGL...
Fetching quote for AMZN...
Fetching quote for TSLA...
Fetching quote for META...
Fetching quote for NFLX...
Fetching quote for NVDA...
Fetching quote for INTC...
Fetching quote for AMD...
  Symbol  Current_Price  Change  Percent_Change  High_Price  Low_Price  \
0   AAPL         207.49   -2.19         -1.0444     211.890     207.11   
1   MSFT         449.78    4.08          0.9154     450.550     446.51   
2  GOOGL         179.63    3.33          1.8888     180.850     176.61   
3   AMZN         189.08    2.98          1.6013     189.275     185.87   
4   TSLA         183.01    1.44          0.7931     183.950     180.69   

   Open_Price  Previous_Close           t                  timestamp  
0     210.390          209.68  1719000002 2024-06-24 10:17:45.040906  
1     447.135          445.70  1719000003 2024-06-24 10:17:45.040906  
2     177.000          176.30  1719000002 2024-06-24 10:17:45.040906  
3

## <b>Transform Data

In [8]:
def transform_stock_data(df):
    # Drop the 't' column
    if 't' in df.columns:
        df = df.drop(columns=['t'])
    
    # Convert 'timestamp' to datetime and extract year, month, day, and time
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['Year'] = df['timestamp'].dt.year
    df['Month'] = df['timestamp'].dt.month
    df['Day'] = df['timestamp'].dt.day
    df['Time'] = df['timestamp'].dt.strftime('%H:%M:%S')
    
    # Drop the original 'timestamp' column
    df = df.drop(columns=['timestamp'])
    
    # Drop rows with NaN values
    df = df.dropna()
    
    return df

# Transform data
transformed_stock_data_df = transform_stock_data(stock_data_df)
print(transformed_stock_data_df.head())

  Symbol  Current_Price  Change  Percent_Change  High_Price  Low_Price  \
0   AAPL         207.49   -2.19         -1.0444     211.890     207.11   
1   MSFT         449.78    4.08          0.9154     450.550     446.51   
2  GOOGL         179.63    3.33          1.8888     180.850     176.61   
3   AMZN         189.08    2.98          1.6013     189.275     185.87   
4   TSLA         183.01    1.44          0.7931     183.950     180.69   

   Open_Price  Previous_Close  Year  Month  Day      Time  
0     210.390          209.68  2024      6   24  10:17:45  
1     447.135          445.70  2024      6   24  10:17:45  
2     177.000          176.30  2024      6   24  10:17:45  
3     187.740          186.10  2024      6   24  10:17:45  
4     182.190          181.57  2024      6   24  10:17:45  


## <b>Load Data

In [10]:
def load_data_to_sqlite(df, db_name='Database/stock_data.db', table_name='stock_quotes'):
    
    # Ensure the database directory exists
    os.makedirs(os.path.dirname(db_name), exist_ok=True)
    
    # Connect to SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create table if not exists
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            Symbol TEXT,
            Current_Price REAL,
            Change REAL,
            Percent_Change REAL,
            High_Price REAL,
            Low_Price REAL,
            Open_Price REAL,
            Previous_Close REAL,
            Year INTEGER,
            Month INTEGER,
            Day INTEGER,
            Time TEXT
        )
    ''')
    
    # Insert data into table
    df.to_sql(table_name, conn, if_exists='append', index=False)
    
    # Commit and close connection
    conn.commit()
    conn.close()

# Load data
load_data_to_sqlite(transformed_stock_data_df)